# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [5]:
import pandas as pd
import pymongo

In [6]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [7]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [15]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors='coerce')
df_youtube = df_youtube.dropna(subset=['date'])
 

df_youtube.rename(columns={'video_id':'_id'}, inplace=True)
 
df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')
 
l_d =df_youtube.to_dict(orient='records')
 
import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)
 
for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [17]:
from pymongo import MongoClient

client = MongoClient("mongo:27017")

db_yt = client.yt

collection_yt = db_yt['videos']

collection_yt.delete_many({})

collection_yt.insert_many(l_d)

## Question 1  

In [20]:
cur = collection_yt.find({"channel_title":"Apple"})
next(cur)

{'_id': 'K4wEI5zhHB0',
 'title': 'iPhone X — Introducing iPhone X — Apple',
 'channel_title': 'Apple',
 'category_id': 28,
 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
 'views': 19707391,
 'likes': 381919,
 'dislikes': 69465,
 'comment_total': 0,
 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
 'date': datetime.datetime(1970, 1, 1, 0, 0),
 'category_name': 'Science & Technology'}

## Question 2

In [40]:
cur = collection_yt.aggregate([{"$group" : {"_id" : "$category_id", "numbers" : {"$sum" : 1}}}])
list(cur)

[{'_id': 28, 'numbers': 151},
 {'_id': 23, 'numbers': 214},
 {'_id': 15, 'numbers': 38},
 {'_id': 19, 'numbers': 18},
 {'_id': 24, 'numbers': 477},
 {'_id': 20, 'numbers': 28},
 {'_id': 1, 'numbers': 101},
 {'_id': 22, 'numbers': 256},
 {'_id': 27, 'numbers': 94},
 {'_id': 2, 'numbers': 37},
 {'_id': 25, 'numbers': 198},
 {'_id': 43, 'numbers': 1},
 {'_id': 29, 'numbers': 5},
 {'_id': 10, 'numbers': 335},
 {'_id': 17, 'numbers': 149},
 {'_id': 26, 'numbers': 262}]

## Question 3

## Question 4

In [47]:
cur = collection_yt.find({}).sort([("views", -1)])
next(cur)

{'_id': 'MBdVXkSdhwU',
 'title': "BTS (방탄소년단) 'DNA' Official MV",
 'channel_title': 'ibighit',
 'category_id': 10,
 'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
 'views': 41500672,
 'likes': 2010366,
 'dislikes': 78076,
 'comment_total': 736179,
 'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
 'date': datetime.datetime(1970, 1, 1, 0, 0),
 'category_name': 'Music'}

## Question 5

## Question 6 

In [50]:
cur = collection_yt.find({}).sort([("likes", -1)])
next(cur)

{'_id': 'MBdVXkSdhwU',
 'title': "BTS (방탄소년단) 'DNA' Official MV",
 'channel_title': 'ibighit',
 'category_id': 10,
 'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
 'views': 41500672,
 'likes': 2010366,
 'dislikes': 78076,
 'comment_total': 736179,
 'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
 'date': datetime.datetime(1970, 1, 1, 0, 0),
 'category_name': 'Music'}